# 개발 환경 및 라이브러리 버전

In [ ]:
import os
import sys
import platform

# OS 정보 출력
print(f"OS: {os.name}") 
print(f"Platform: {platform.system()} {platform.release()}") 

# Python 버전 출력
print(f"Python Version: {sys.version}") 
print(f"Python Version (short): {platform.python_version()}") 

In [ ]:
import importlib

# 확인할 라이브러리 목록
libraries = [
    "os", "time", "gc", "pickle", "joblib", "collections",  # 기본 라이브러리
    "numpy", "pandas",  # 데이터 처리
    "matplotlib", "tqdm",  # 시각화 및 진행바
    "sklearn", "skopt",  # 머신러닝 관련
    "xgboost", "catboost",  # 머신러닝 모델
    "torch", "transformers"  # 딥러닝 관련
]

# 버전 확인 함수
def get_library_versions(libs):
    versions = {}
    for lib in sorted(libs):
        try:
            module = importlib.import_module(lib)
            version = getattr(module, '__version__', 'Unknown')
            versions[lib] = version
        except ImportError:
            versions[lib] = "Not Installed"
        except AttributeError:
            versions[lib] = "No Version Info"

    return versions

versions = get_library_versions(libraries)

# Python 버전 출력
print(f"Python Version: {sys.version.split()[0]}")

# 라이브러리 버전 출력
for lib, version in versions.items():
    print(f"{lib}: {version}")

# Import

In [ ]:
import os
import time
import gc
import pickle
import joblib
from collections import OrderedDict

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
plt.rc('font', family='NanumGothic')  # 한글 깨짐 방지 (Linux 사용자)
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지
from tqdm import tqdm

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score, make_scorer

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.model_selection import GridSearchCV
from skopt import BayesSearchCV

from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from transformers import (
    BertTokenizerFast,
    DistilBertForSequenceClassification,
    AdamW,
    get_scheduler
)

# 0. 디렉토리 설정

In [ ]:
dir = "./datasets/"               # raw 데이터 & 전처리 완료된 데이터 & sample submission이 저장된 디렉토리
dir_to_submit = "./submit/"       # submission 파일들이 저장될 디렉토리
dir_models = './models/'          # model.pkl 파일들이 저장될 디렉토리

# 폴더가 없다면 생성
os.makedirs(dir, exist_ok=True)
os.makedirs(dir_to_submit, exist_ok = True)
os.makedirs(dir_models, exist_ok = True)

print(f"폴더 '{dir}, {dir_to_submit}, {dir_models}'생성")

# 1. Preprocessing

In [ ]:
import sys
print(sys.version)

In [ ]:
import pkg_resources
for pkg in sorted(pkg_resources.working_set, key=lambda x: x.project_name.lower()):
    print(f"{pkg.project_name}=={pkg.version}")

## Data Load

In [ ]:
train = pd.read_csv(dir + 'train.csv', encoding='utf-8').drop(columns=['ID'])
test = pd.read_csv(dir + 'test.csv', encoding='utf-8').drop(columns=['ID'])

## 1.1. 변수 변형

### 1.1.1. `배아 생성 주요 이유` 컬럼
- 현재 시술용, 난자 저장용, 배아 저장용, 기증용 칼럼을 만들어 해당 값이 존재하면 1, 존재하지 않으면 0
- 기존 칼럼은 삭제

In [ ]:
def process_embryo_reason(train_df, test_df):
    # 데이터 복사
    train_df = train_df.copy()
    test_df = test_df.copy()

    # 새로운 컬럼 리스트
    new_columns = ['현재 시술용', '난자 저장용', '배아 저장용', '기증용']

    # 각 컬럼을 0으로 초기화
    if '배아 생성 주요 이유' in train_df.columns:
        for col in new_columns:
            train_df[col] = 0
            test_df[col] = 0


        for index, value in train_df['배아 생성 주요 이유'].items():
            if pd.notna(value):  # 결측값이 아닌 경우만 처리
                for col in new_columns:
                    if col in str(value):  # 문자열 변환 후 확인
                        train_df.at[index, col] = 1


        for index, value in test_df['배아 생성 주요 이유'].items():
            if pd.notna(value):  # 결측값이 아닌 경우만 처리
                for col in new_columns:
                    if col in str(value):  # 문자열 변환 후 확인
                        test_df.at[index, col] = 1

    # 기존 '배아 생성 주요 이유' 컬럼 삭제
    train_df.drop(columns=['배아 생성 주요 이유'], inplace=True, errors='ignore')
    test_df.drop(columns=['배아 생성 주요 이유'], inplace=True, errors='ignore')

    return train_df, test_df

# 사용 예시
train, test = process_embryo_reason(train, test)

## 1.2. 이상치 처리

### 1.2.1. 배아 관련 이상치(IVF)
- “`특정 시술 유형`”에 ICSI가 포함되지 않는데 “미세”가 포함된 변수 값이 모두 0이 아닌 행 삭제
- “`이식된 배아 수`” == 0인데 “`배아 이식 경과일`” > 0 인 행 삭제
- “`동결 배아 사용 여부`” == 0인데 “`해동된 배아 수`” > 0인 행 삭제
- “`동결 배아 사용 여부`” == 0 & “`배아 해동 경과일`”이 결측일 때 “`해동된 배아 수`” ≠ 0 인 행 삭제

In [ ]:
def remove_embryo_outliers(train_df):
    """
    배아 관련 이상치 데이터를 제거하는 함수.
    
    - 특정 시술 유형이 IVF이면서, 특정 시술 유형에 ICSI가 포함되지 않는데 미세 관련 변수가 0이 아닌 행 삭제
    - 특정 시술 유형이 IVF이면서, 이식된 배아 수가 0인데 배아 이식 경과일 > 0 인 행 삭제
    - 특정 시술 유형이 IVF이면서, 동결 배아 사용 여부가 0인데 해동된 배아 수 > 0인 행 삭제
    - 특정 시술 유형이 IVF이면서, 동결 배아 사용 여부가 0 & 배아 해동 경과일이 결측인데 해동된 배아 수 ≠ 0 인 행 삭제
    """

    # 삭제 전 데이터 크기 저장
    train_size_before = len(train_df)

    # 1. 특정 시술 유형이 IVF이면서, 특정 시술 유형에 ICSI가 없는데 "미세" 포함 변수 값이 0이 아닌 행 삭제
    micro_columns = [col for col in train_df.columns if "미세" in col]  # "미세" 포함된 컬럼 찾기
    condition = (train_df["시술 유형"] == "IVF") & (~train_df["특정 시술 유형"].str.contains("ICSI", na=False)) & (train_df[micro_columns].sum(axis=1) > 0)
    train_df = train_df[~condition]

    # 2. 특정 시술 유형이 IVF이면서, 이식된 배아 수 == 0인데 배아 이식 경과일 > 0 인 행 삭제
    condition = (train_df["시술 유형"] == "IVF") & (train_df["이식된 배아 수"] == 0) & (train_df["배아 이식 경과일"] > 0)
    train_df = train_df[~condition]

    # 3. 특정 시술 유형이 IVF이면서, 동결 배아 사용 여부 == 0인데 해동된 배아 수 > 0 인 행 삭제
    condition = (train_df["시술 유형"] == "IVF") & (train_df["동결 배아 사용 여부"] == 0) & (train_df["해동된 배아 수"] > 0)
    train_df = train_df[~condition]

    # 4. 특정 시술 유형이 IVF이면서, 동결 배아 사용 여부 == 0 & 배아 해동 경과일이 결측인데 해동된 배아 수 ≠ 0 인 행 삭제
    condition = (train_df["시술 유형"] == "IVF") & (train_df["동결 배아 사용 여부"] == 0) & (train_df["배아 해동 경과일"].isna()) & (train_df["해동된 배아 수"] != 0)
    train_df = train_df[~condition]

    # 삭제 후 데이터 크기 저장
    train_size_after = len(train_df)

    # 삭제된 행 수 계산
    train_removed = train_size_before - train_size_after
    # 삭제된 행 수 출력
    print(f"Train 데이터에서 삭제된 행 수: {train_removed}")

    return train_df

train = remove_embryo_outliers(train)

### 1.2.2. '`동결 배아 사용 여부`' == 0, '`해동된 배아 수`' == 0, '`배아 해동 경과일`' != 0 인 행들을 삭제

In [ ]:
def remove_invalid_embryo_rows(train_df):
    """
    특정 시술 유형이 IVF인 경우,
    '동결 배아 사용 여부' == 0, '해동된 배아 수' == 0, '배아 해동 경과일'이 NaN이 아닌 경우의 행을 삭제하는 함수.
    """

    # 삭제 전 데이터 크기 저장
    train_size_before = len(train_df)

    # 특정 시술 유형이 IVF인 경우만 적용
    condition = (train_df['시술 유형'] == "IVF") & \
                (train_df['동결 배아 사용 여부'] == 0) & \
                (train_df['해동된 배아 수'] == 0) & \
                (pd.notna(train_df['배아 해동 경과일']))  # NaN이 아닌 경우 필터링
    train_df = train_df[~condition]  # 조건에 맞는 행 삭제

    # 삭제된 행 수 계산
    train_removed = train_size_before - len(train_df)

    # 삭제된 행 수 출력
    print(f"Train 데이터에서 삭제된 행 수: {train_removed}")

    return train_df

# 함수 사용 예시
train = remove_invalid_embryo_rows(train)

### 1.2.3. `특정 시술 유형` 컬럼 결측인 행 제거

In [ ]:
def remove_outliers_procedure(train_df):
    """ 
    '특정 시술 유형' 컬럼이 존재할 경우, 결측(NaN)인 행을 제거하고, 제거된 행 수를 출력하는 함수.
    컬럼이 없으면 경고 메시지를 출력하고 원래 데이터프레임을 반환.
    """

    # 특정 시술 유형 컬럼 존재 여부 확인
    if '특정 시술 유형' not in train_df.columns:
        print("⚠️ '특정 시술 유형' 컬럼이 데이터프레임에 존재하지 않습니다. 원본 데이터를 반환합니다.")
        return train_df

    # 원본 데이터 크기 저장
    train_size_before = len(train_df)

    # '특정 시술 유형'이 NaN인 행 제거
    train_df = train_df.dropna(subset=['특정 시술 유형'])

    # 제거된 행 수 계산
    train_removed = train_size_before - len(train_df)

    # 제거된 행 수 출력
    print(f"Train 데이터에서 제거된 행 수: {train_removed}")

    return train_df
train = remove_outliers_procedure(train)

## 1.3. 결측 처리

### 1.3.1. IVF 시술을 받은 환자의 경우 결측처리
- IVF 시술을 받은 환자의 경우 아래와 같이 결측 처리
- '`이식된 배아 수`'가 0이면 '`배아 이식 경과일`'의 결측값을 0으로 채움.
- '`동결 배아 사용 여부`'가 0이면 '`배아 해동 경과일`'의 결측값을 0으로 채움.

In [ ]:
def fill_missing_embryo_days(train_df, test_df):
    """
    IVF 시술을 받은 환자의 경우,
    - '이식된 배아 수'가 0이면 '배아 이식 경과일'의 결측값을 0으로 채움.
    - '동결 배아 사용 여부'가 0이면 '배아 해동 경과일'의 결측값을 0으로 채움.
    """

    def fill_embryo_days(df):
        # IVF 시술을 받은 경우만 적용
        ivf_mask = df['시술 유형'] == "IVF"

        # '이식된 배아 수' == 0 → '배아 이식 경과일' NaN을 0으로 채우기
        condition_1 = ivf_mask & (df['이식된 배아 수'] == 0) & (df['배아 이식 경과일'].isna())
        df.loc[condition_1, '배아 이식 경과일'] = 0

        # '동결 배아 사용 여부' == 0 → '배아 해동 경과일' NaN을 0으로 채우기
        condition_2 = ivf_mask & (df['동결 배아 사용 여부'] == 0) & (df['배아 해동 경과일'].isna())
        df.loc[condition_2, '배아 해동 경과일'] = 0

        # 변경된 데이터 개수 출력
        print(f"[데이터셋: {df.name}] '배아 이식 경과일' 채운 개수: {condition_1.sum()}")
        print(f"[데이터셋: {df.name}] '배아 해동 경과일' 채운 개수: {condition_2.sum()}")

        return df

    # DataFrame 이름 추가 (출력 시 구분하기 위해)
    train_df.name = "Train"
    test_df.name = "Test"

    train_df = fill_embryo_days(train_df)
    test_df = fill_embryo_days(test_df)

    return train_df, test_df


#   함수 실행
train, test = fill_missing_embryo_days(train, test)

### 1.3.2. "`특정 시술 유형`' 컬럼 결측 처리
- "Unknown" => nan으로 변경
- Unknown이 : 으로 묶인 경우 => Unknown 칼럼 생성
- "FER", "GIFT" 는Unknown으로

In [ ]:
def encode_treatment_types(train_df, test_df):
    # 데이터 복사
    train_df = train_df.copy()
    test_df = test_df.copy()
    
    train_df["특정 시술 유형"] = train_df['특정 시술 유형'].replace("FER", "Unknown").replace("GIFT", "Unknown")
    test_df["특정 시술 유형"] = test_df['특정 시술 유형'].replace("FER", "Unknown").replace("GIFT", "Unknown")

    # 특정 시술 유형에서 고유한 시술 종류 추출
    all_treatments = set()

    for df in [train_df, test_df]:
        if '특정 시술 유형' in df.columns:
            df['특정 시술 유형'] = df['특정 시술 유형'].astype(str).fillna("Unknown")  # 결측값 방지
            df['특정 시술 유형'].str.replace(" ", "").str.split(":|/").apply(all_treatments.update)
    
    # 'nan' 값이 존재하면 제거
    all_treatments.discard('nan')
    
    # 새로운 시술 유형 컬럼 생성 (초기값 0)
    for treatment in all_treatments:
        train_df[treatment] = 0
        test_df[treatment] = 0

    # 해당 시술이 포함된 경우 1로 설정
    for df in [train_df, test_df]:
        if '특정 시술 유형' in df.columns:
            for treatment in all_treatments:
                df[treatment] = df['특정 시술 유형'].str.replace(" ", "").apply(lambda x: 1 if treatment in x else 0)

    # 기존 '특정 시술 유형' 컬럼 삭제
    train_df.drop(columns=['특정 시술 유형'], inplace=True, errors='ignore')
    test_df.drop(columns=['특정 시술 유형'], inplace=True, errors='ignore')

    # 추가된 칼럼 목록 출력
    print(f"추가된 시술 유형 칼럼들: {list(all_treatments)}")
    return train_df, test_df

# 사용 예시
train, test = encode_treatment_types(train, test)

## 1.4. Ordinal Encoding
- 횟수 변수들 ordinal encoding
- 총 ~ 횟수는 IVF ~ 횟수 + DI ~ 횟수로 추정

In [ ]:
def ordinal_encoding(train_df, test_df):
    """
    나이 관련 변수를 순서형 숫자로 변환하고, 빈도 관련 변수를 정수 변환하는 함수.
    """

    # 데이터 복사
    train_df = train_df.copy()
    test_df = test_df.copy()


    # 빈도 관련 컬럼 변환 (존재하는 컬럼만 처리)
    freq_columns = ['총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', 
                    '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수']

    for col in freq_columns:
        if col in train_df.columns:
            train_df[col] = train_df[col].apply(lambda x: int(str(x)[:1]) if pd.notna(x) and str(x)[0].isdigit() else x)
            test_df[col] = test_df[col].apply(lambda x: int(str(x)[:1]) if pd.notna(x) and str(x)[0].isdigit() else x)
    
    train_df['총 시술 횟수'] = train_df['IVF 시술 횟수'] + train_df['DI 시술 횟수']
    test_df['총 시술 횟수'] = test_df['IVF 시술 횟수'] + test_df['DI 시술 횟수']
    
    train_df['총 임신 횟수'] = train_df['IVF 임신 횟수'] + train_df['DI 임신 횟수']
    test_df['총 임신 횟수'] = test_df['IVF 임신 횟수'] + test_df['DI 임신 횟수']
    
    train_df['총 출산 횟수'] = train_df['IVF 출산 횟수'] + train_df['DI 출산 횟수']
    test_df['총 출산 횟수'] = test_df['IVF 출산 횟수'] + test_df['DI 출산 횟수']

    return train_df, test_df

# 사용 예시
train, test = ordinal_encoding(train, test)

## 1.5. 분산 0인 칼람 삭제

In [ ]:
def remove_single_value_columns(train_df, test_df, fill_value=999):
    """
    NaN을 임시 값으로 채운 후, 유일한 값이 하나뿐인 컬럼을 삭제하는 함수

    """

    # NaN을 임시 값으로 채운 후, 유일한 값이 하나뿐인 컬럼 찾기
    single_value_cols_train = train_df.fillna(fill_value).nunique() == 1

    # 제거할 컬럼 리스트
    cols_to_drop = train_df.columns[single_value_cols_train].tolist()

    # 데이터에서 해당 컬럼 삭제
    train_df = train_df.drop(columns=cols_to_drop)
    test_df = test_df.drop(columns=cols_to_drop)

    # 삭제된 컬럼 출력
    print(f"삭제된 단일 값 컬럼: {cols_to_drop}")

    return train_df, test_df


train, test = remove_single_value_columns(train, test)


## 1.6. 파생 변수
아래 파생 변수 추가
- 시술 실패 관련
    - `총 시술 실패 횟수` = 총 시술 횟수 - 총 임신 횟수
    - `IVF 시술 실패 횟수` = IVF 시술 횟수 - IVF 임신 횟수
    - `DI 시술 실패 횟수` = DI 시술 횟수 - DI 임신 횟수

- 출산 실패 관련
    - `총 출산 실패 횟수` = 총 임신 횟수 - 총 출산 횟수
    - `IVF 출산 실패 횟수` = IVF 임신 횟수 - IVF 출산 횟수
    - `DI 출산 실패 횟수` = DI 임신 횟수 - DI 출산 횟수

- 시술 성공률 관련
    - `총 임신 성공률` = 총 임신 횟수 / 총 시술 횟수
    - `IVF 임신 성공률` = IVF 임신 횟수 / IVF 시술 횟수
    - `DI 임신 성공률` = DI 임신 횟수 / DI 시술 횟수

- 출산 성공률 관련
    - `총 출산 성공률` = 총 출산 횟수 / 총 임신 횟수
    - `IVF 출산 성공률` = IVF 출산 횟수 / IVF 임신 횟수
    - `DI 출산 성공률` = DI 출산 횟수 / DI 임신 횟수

In [ ]:
def add_failure_counts(train_df, test_df):
    """
    시술 실패 횟수 및 출산 실패 횟수를 계산하여 추가하는 함수.
    """

    # 시술 실패 횟수 계산
    train_df["총 시술 실패 횟수"] = train_df["총 시술 횟수"] - train_df["총 임신 횟수"]
    train_df["IVF 시술 실패 횟수"] = train_df["IVF 시술 횟수"] - train_df["IVF 임신 횟수"]
    train_df["DI 시술 실패 횟수"] = train_df["DI 시술 횟수"] - train_df["DI 임신 횟수"]

    test_df["총 시술 실패 횟수"] = test_df["총 시술 횟수"] - test_df["총 임신 횟수"]
    test_df["IVF 시술 실패 횟수"] = test_df["IVF 시술 횟수"] - test_df["IVF 임신 횟수"]
    test_df["DI 시술 실패 횟수"] = test_df["DI 시술 횟수"] - test_df["DI 임신 횟수"]

    # 출산 실패 횟수 계산
    train_df["총 출산 실패 횟수"] = train_df["총 임신 횟수"] - train_df["총 출산 횟수"]
    train_df["IVF 출산 실패 횟수"] = train_df["IVF 임신 횟수"] - train_df["IVF 출산 횟수"]
    train_df["DI 출산 실패 횟수"] = train_df["DI 임신 횟수"] - train_df["DI 출산 횟수"]

    test_df["총 출산 실패 횟수"] = test_df["총 임신 횟수"] - test_df["총 출산 횟수"]
    test_df["IVF 출산 실패 횟수"] = test_df["IVF 임신 횟수"] - test_df["IVF 출산 횟수"]
    test_df["DI 출산 실패 횟수"] = test_df["DI 임신 횟수"] - test_df["DI 출산 횟수"]

    return train_df, test_df


train, test = add_failure_counts(train, test)

In [ ]:
def add_success_rate_and_treatment_experience(train_df, test_df):
    """
    성공률 및 시술 경험 여부 변수를 추가하는 함수.
    """

    #   성공률 계산을 위한 함수 (0으로 나누는 경우 방지)
    def compute_success_rate(numerator, denominator):
        return numerator / (denominator + 1)  #   시술 횟수가 0인 경우 방지

    #   성공률 변수 추가
    for df in [train_df, test_df]:
        df["총 임신 성공률"] = compute_success_rate(df["총 임신 횟수"], df["총 시술 횟수"])
        df["IVF 임신 성공률"] = compute_success_rate(df["IVF 임신 횟수"], df["IVF 시술 횟수"])
        df["DI 임신 성공률"] = compute_success_rate(df["DI 임신 횟수"], df["DI 시술 횟수"])
        df["총 출산 성공률"] = compute_success_rate(df["총 출산 횟수"], df["총 임신 횟수"])
        df["IVF 출산 성공률"] = compute_success_rate(df["IVF 출산 횟수"], df["IVF 임신 횟수"])
        df["DI 출산 성공률"] = compute_success_rate(df["DI 출산 횟수"], df["DI 임신 횟수"])

        #   시술 경험 여부 변수 추가
        df["전체 시술 경험 여부"] = (df["총 시술 횟수"] > 0).astype(int)   # 1회 이상 시술한 경우 1, 아니면 0
        df["IVF 시술 경험 여부"] = (df["IVF 시술 횟수"] > 0).astype(int)   # IVF 시술 경험 여부
        df["DI 시술 경험 여부"] = (df["DI 시술 횟수"] > 0).astype(int)     # DI 시술 경험 여부

    return train_df, test_df

#   함수 실행
train, test = add_success_rate_and_treatment_experience(train, test)

-  `자연 수정 배아 비율` 

    IVF 시술을 받은 환자의 경우,

    - '자연 수정 배아 비율' = 1 - (미세 주입 배아 이식 수 / (이식된 배아 수 + 1)) 변수 추가
    - DI 시술을 받은 환자의 경우, 해당 변수를 NaN으로 설정
    


In [ ]:
def add_natural_fertilization_rate(train_df, test_df):
    """
    IVF 시술을 받은 환자의 경우,
    - '자연 수정 배아 비율' = 1 - (미세 주입 배아 이식 수 / (이식된 배아 수 + 1)) 변수 추가
    - DI 시술을 받은 환자의 경우, 해당 변수를 NaN으로 설정
    
    train_df와 test_df 모두 동일한 로직을 적용함.
    """

    def compute_natural_fertilization(df):
        #   IVF 시술을 받은 경우만 계산
        ivf_mask = df['시술 유형'] == "IVF"

        #   자연 수정 배아 비율 계산 (IVF만 적용)
        df.loc[ivf_mask, '자연 수정 배아 비율'] = 1 - (df['미세주입 배아 이식 수'] / (df['이식된 배아 수'] + 1))

        #   DI 시술을 받은 경우 NaN 설정
        di_mask = df['시술 유형'] == "DI"
        df.loc[di_mask, '자연 수정 배아 비율'] = np.nan

        #   변경된 데이터 개수 출력
        print(f" [데이터셋: {df.name}] IVF 자연 수정 배아 비율 추가 개수: {ivf_mask.sum()}")
        print(f" [데이터셋: {df.name}] DI 자연 수정 배아 비율 NaN 처리 개수: {di_mask.sum()}")

        return df

    #   DataFrame 이름 추가 (출력 시 구분하기 위해)
    train_df.name = "Train"
    test_df.name = "Test"

    train_df = compute_natural_fertilization(train_df)
    test_df = compute_natural_fertilization(test_df)

    return train_df, test_df


#   함수 실행
train, test = add_natural_fertilization_rate(train, test)

## 1.7. 추가 결측 처리
`배아 이식 경과일` 컬럼에 IterativeImputer 적용하여 추가적으로 결측 대체

In [ ]:
def mice_embryo_days(train_df, test_df):
    # IVF 시술 유형 데이터 필터링
    train_ivf = train_df[train_df["시술 유형"] == "IVF"].copy()
    test_ivf = test_df[test_df["시술 유형"] == "IVF"].copy()
    
    # 사용할 변수 목록 (타겟 + 상관관계 높은 변수)
    features = [
        "배아 이식 경과일", "배란 자극 여부", "총 시술 횟수", "IVF 시술 횟수",
        "총 생성 배아 수", "미세주입된 난자 수", "미세주입에서 생성된 배아 수",
        "저장된 배아 수", "해동된 배아 수", "수집된 신선 난자 수", "혼합된 난자 수",
        "파트너 정자와 혼합된 난자 수", "동결 배아 사용 여부", "신선 배아 사용 여부",
        "난자 채취 경과일"
    ]

    # MICE 적용할 데이터셋 추출
    train_mice = train_ivf[features]
    test_mice = test_ivf[features]

    # MICE 모델 학습 (train 데이터만 사용)
    # imputer = IterativeImputer(estimator=RandomForestRegressor(n_estimators=100), max_iter=10, random_state=42)
    mice_imputer = IterativeImputer(max_iter=10, random_state=42)
    train_imputed = mice_imputer.fit_transform(train_mice)

    # 훈련된 MICE 모델로 test 데이터 결측 보강
    test_imputed = mice_imputer.transform(test_mice)

    # 보강된 값 적용 (DataFrame으로 변환)
    train_ivf_imputed = pd.DataFrame(train_imputed, columns=features, index=train_ivf.index)
    test_ivf_imputed = pd.DataFrame(test_imputed, columns=features, index=test_ivf.index)

    # 0보다 작은 값은 0으로 변경, 7 이상인 값은 7로 제한 후, 정수로 반올림
    train_ivf_imputed["배아 이식 경과일"] = train_ivf_imputed["배아 이식 경과일"].clip(lower=0, upper=10)
    test_ivf_imputed["배아 이식 경과일"] = test_ivf_imputed["배아 이식 경과일"].clip(lower=0, upper=10)

    # 원본 데이터에 보강된 값 반영 (DI 시술 데이터는 변경 없음)
    train_df.loc[train_ivf.index, "배아 이식 경과일"] = train_ivf_imputed["배아 이식 경과일"]
    test_df.loc[test_ivf.index, "배아 이식 경과일"] = test_ivf_imputed["배아 이식 경과일"]
    
    return train_df, test_df

train, test = mice_embryo_days(train, test)

## 1.8. 연속형 변수 범주형 변환
- `해동 난자 수 컬럼` 을 구간을 나누어 범주형으로 변환

In [ ]:
def convert_numeric_to_categorical(train_df, test_df, bin_cols):
    """
    특정 칼럼들의 값을 구간을 나눠 범주형으로 변환하는 함수.
    """
    #   범주화할 구간 설정 (0은 그대로 두고, 이후 5단위 구간 생성)
    bins = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 25, 30, 40, 50, 100]  # 원하는 범위 추가 가능
    labels = [f"{bins[i]+1}-{bins[i+1]}" for i in range(len(bins)-1)]  # "1-5", "6-10" 같은 라벨 자동 생성

    for df in [train_df, test_df]:
        for col in bin_cols:
            df[col] = pd.cut(df[col], bins=[-1] + bins, labels=["0"] + labels).astype(str)  #   -1을 추가해 0 유지

    return train_df, test_df

#   함수 실행 예시
bin_cols = ["해동 난자 수"]  # 변환할 변수 리스트
train, test = convert_numeric_to_categorical(train, test, bin_cols)

In [ ]:
train.shape

Preprocessing data 저장

In [ ]:
train.to_csv(dir + 'train_Preprocessed.csv', encoding = 'utf-8')
test.to_csv(dir + 'test_Preprocessed.csv', encoding = 'utf-8')
print(f'train_Preprocessed.csv, test_Preprocessed.csv {dir}에 저장완료')

-----

# 2. CatBoost

In [ ]:
# 수정된 컬럼명으로 X, y 분리
X = train.drop(columns=['임신 성공 여부'])
y = train['임신 성공 여부']


# X에서 object 타입(범주형)인 컬럼 찾기
cat_columns = X.select_dtypes(include=['object']).columns.tolist()
X[cat_columns] = X[cat_columns].astype(str)
test[cat_columns] = test[cat_columns].astype(str)

# 결과 확인
print(" 범주형 변수 목록:", cat_columns)

## 2.1. Hyperparameter Tunning

### 2.1.1. Gridsearch
- params
    - `N_JOBS` : 모델 훈련에 사용할 CPU 코어 수
    - `cv_seed` : cross-validation random seed
    - `n_cv` : cross-validation seed 수 
    - `param_grid` : grid setting
    - `models` : 모델 정의

In [ ]:
# 병렬 처리 설정
N_JOBS = 1  # CPU 코어 개수 (변경 가능)
cv_seed = 5534  # n_cv fold cv 의 시드
n_cv = 3
n_iter = 1 # Bayes Search 횟수

param_grid = {
    "CatBoost": {
        "border_count" : [64],
        "depth": [7], 
        "learning_rate": [0.032250131659519184], 
        "iterations": [920],
        "l2_leaf_reg" : [145],
        "scale_pos_weight" : [3.510614653412628]    
    }
}
############################## 최적값 ##############################

# 모델 정의
models = {
    "CatBoost": CatBoostClassifier(
                verbose=0, 
                random_state=42,
                task_type="GPU",
                cat_features=cat_columns,

                )
}

In [ ]:
#  CatBoost 전용 Pool 객체 생성
train_pool = Pool(data=X, label=y, cat_features=cat_columns)

# AUC 점수를 소수점 6자리까지 출력하는 custom scorer
def custom_auc_scorer(y_true, y_pred):
    auc_score = roc_auc_score(y_true, y_pred)
    rounded_score = round(auc_score, 6)  # 소수점 6자리 반올림
    print(f" Fold AUC: {rounded_score}")  # Fold마다 출력
    return auc_score

# 결과 저장
best_models = {}
cv_results = {}
auc_results = {}

start_time = time.time()  

for model_name, model in models.items():
    print(f"\n {model_name} 모델 GridSearch 시작...")
    model_start = time.time()  

    cv = StratifiedKFold(n_splits=n_cv, shuffle=True, random_state=cv_seed)
    
    grid_search = GridSearchCV(
    model, 
    param_grid[model_name], 
    scoring=make_scorer(custom_auc_scorer, response_method="predict_proba"),  
    cv=cv, 
    n_jobs=N_JOBS,  #  병렬 처리
    verbose=n_cv
    )

    
    #   `cat_features`를 GridSearch에도 적용 (astype('category') 변환)
    X_catboost = X.copy()
    X_catboost[cat_columns] = X_catboost[cat_columns].astype('category')  #  범주형 변수 변환
    
    #   GridSearchCV의 fit()에서 cat_features 명시적으로 전달
    grid_search.fit(X_catboost, y)  

    best_models[model_name] = grid_search.best_estimator_
    print(f"  {model_name} 최적 파라미터: {grid_search.best_params_}")

    print(f" {model_name} {n_cv}-Fold Cross Validation 시작...")
    train_auc_scores = []
    val_auc_scores = []

    for fold_idx, (train_idx, val_idx) in tqdm(enumerate(cv.split(X, y), 1), total=n_cv, desc=f" {model_name} CV Progress"):
        X_t, X_v = X.iloc[train_idx], X.iloc[val_idx]
        y_t, y_v = y.iloc[train_idx], y.iloc[val_idx]

        #   학습 & 검증용 Pool 생성 (범주형 변수 설정)
        train_fold_pool = Pool(data=X_t, label=y_t, cat_features=cat_columns)
        val_fold_pool = Pool(data=X_v, label=y_v, cat_features=cat_columns)

        model = grid_search.best_estimator_
        model.fit(train_fold_pool)  #  `Pool`을 사용하여 학습

        #   Train AUC 계산
        y_train_pred = model.predict_proba(train_fold_pool)[:, 1]
        train_auc = roc_auc_score(y_t, y_train_pred)
        train_auc_scores.append(round(train_auc, 6))

        #   Validation AUC 계산
        y_val_pred = model.predict_proba(val_fold_pool)[:, 1]
        val_auc = roc_auc_score(y_v, y_val_pred)
        val_auc_scores.append(round(val_auc, 6))



    print(f"\n {model_name} {n_cv}-Fold Train AUC Scores = {train_auc_scores}")
    print(f" {model_name} {n_cv}-Fold Validation AUC Scores = {val_auc_scores}")

    mean_train_auc = np.mean(train_auc_scores)
    std_train_auc = np.std(train_auc_scores)

    mean_val_auc = np.mean(val_auc_scores)
    std_val_auc = np.std(val_auc_scores)

    cv_results[model_name] = {"Train AUC": (mean_train_auc, std_train_auc), "Validation AUC": (mean_val_auc, std_val_auc)}
    auc_results[model_name] = {"Train": train_auc_scores, "Validation": val_auc_scores}

    print(f" {model_name} Train AUC: 평균={mean_train_auc:.6f}, 표준편차={std_train_auc:.6f}")
    print(f" {model_name} Validation AUC: 평균={mean_val_auc:.6f}, 표준편차={std_val_auc:.6f}")

    model_end = time.time()
    print(f" {model_name} 학습 완료! 소요 시간: {model_end - model_start:.2f}초\n")

total_time = time.time() - start_time
print(f"\n 전체 학습 완료! 총 소요 시간: {total_time:.2f}초")

### Best params

In [ ]:
print("\n 최적 모델 및 AUC 결과 정리")

#   CV 설정 정보 출력
print(f"cv random_seed: {cv_seed}, n_folds: {n_cv}")

#   CatBoost 최적 하이퍼파라미터 출력
best_catboost_params = best_models["CatBoost"].get_params()
print("\n Best CatBoost Hyperparameters:")
for key, value in best_catboost_params.items():
    print(f"  {key}: {value}")

#   Train & Validation AUC 결과 출력
mean_train_auc, std_train_auc = cv_results["CatBoost"]["Train AUC"]
mean_val_auc, std_val_auc = cv_results["CatBoost"]["Validation AUC"]

print(f"\n {n_cv}-Fold Train AUC Scores: {auc_results['CatBoost']['Train']} (std: {std_train_auc:.6f})")
print(f" {n_cv}-Fold Validation AUC Scores: {auc_results['CatBoost']['Validation']} (std: {std_val_auc:.6f})")

print(f"\n Final Train AUC: 평균={mean_train_auc:.6f}, 표준편차={std_train_auc:.6f}")
print(f" Final Validation AUC: 평균={mean_val_auc:.6f}, 표준편차={std_val_auc:.6f}")

## 2.2. Finalize

In [ ]:
# 7️⃣ 최적 하이퍼파라미터로 최종 모델 학습 (전체 데이터 사용)
print("\n 최적 하이퍼파라미터로 Final Model 학습 시작...")

#   최적 하이퍼파라미터 가져오기
best_params = best_models["CatBoost"].get_params()
best_params.update({"verbose": 0, "random_state": 42, "thread_count": N_JOBS})  #   불필요한 verbose 제거
print(f"finalized params : {best_params}")

#   최종 학습 데이터 Pool 생성 (범주형 변수 포함)
train_pool = Pool(data=X, label=y, cat_features=cat_columns)

#   최종 모델 훈련 (Pool 사용)
final_model = CatBoostClassifier(**best_params)
final_model.fit(train_pool)

print("  Final Model 학습 완료!")
# 0.743

In [ ]:
#  최종 모델 저장
joblib.dump(final_model, dir_models + "final_cat_model.pkl")
print(f" 최적 CatBoost 모델이 `final_cat_model.pkl`로 저장되었습니다!")

## 2.3. predict

### 2.3.1. test predict

In [ ]:
# 8️⃣ Test 데이터에 대해 predict_proba 수행 (Pool 사용)
print("\n Test 데이터 예측 (predict_proba) 시작...")

#   Test 데이터 Pool 생성
X_test = test.copy()
test_pool = Pool(data=X_test, cat_features=cat_columns)  #   Pool 생성

#   예측 수행 (Pool 사용)
test_preds = final_model.predict_proba(test_pool)[:, 1]  #   NumPy 배열 반환

pd.DataFrame(test_preds).to_csv(dir_to_submit + 'test_pred_cat.csv', encoding = 'utf-8')

### 2.3.2. train predict

In [ ]:
t_pool = Pool(data = X, cat_features= cat_columns)
train_preds = final_model.predict_proba(t_pool)[:, 1]

t_prob = pd.DataFrame(train_preds, train['임신 성공 여부'])

# 제출할 파일명
file_name = dir_to_submit + "train_pred_cat.csv"
t_prob.to_csv(file_name, encoding = 'utf-8')

----

# 3. XgBoost

## 3.1. 데이터 로드

In [ ]:
train = pd.read_csv(dir + "train_Preprocessed.csv", encoding='utf-8', index_col = 0)
test = pd.read_csv(dir + "test_Preprocessed.csv", encoding='utf-8', index_col = 0)

## 3.2. 수치형, 범주형 칼럼 분리

In [ ]:
numeric_columns = [ "총 시술 횟수", "클리닉 내 총 시술 횟수", "IVF 시술 횟수", "DI 시술 횟수",
                   "총 임신 횟수", "IVF 임신 횟수", "DI 임신 횟수", "총 출산 횟수", "IVF 출산 횟수", "DI 출산 횟수", "총 생성 배아 수",
                  "미세주입에서 생성된 배아 수", "미세주입 배아 이식 수", "저장된 배아 수", "미세주입 후 저장된 배아 수",
                   "저장된 신선 난자 수",  "기증자 정자와 혼합된 난자 수",
                   "난자 혼합 경과일", "배아 이식 경과일", "배아 해동 경과일", "총 시술 실패 횟수", "IVF 시술 실패 횟수", "DI 시술 실패 횟수", "총 출산 실패 횟수",
                   "IVF 출산 실패 횟수", "DI 출산 실패 횟수", 
                   "총 임신 성공률", "IVF 임신 성공률", "DI 임신 성공률", "총 출산 성공률", "IVF 출산 성공률", "DI 출산 성공률",
                   "자연 수정 배아 비율",
                    # "해동 난자 수",    # 범주형으로 두는게 스코어 더 높음
                    "수집된 신선 난자 수",
                    "혼합된 난자 수",
                    "파트너 정자와 혼합된 난자 수",
                    "이식된 배아 수",   # 이건 수치형이 좋음.
                    "해동된 배아 수",   # 이것도 수치형이 좋음.
                    "임신 시도 또는 마지막 임신 경과 연수", # 이것도 수치형
                    "미세주입된 난자 수",
                   ]

def convert_to_categorical(train_df, test_df, numeric_columns):
    """
    특정 numeric_columns를 제외한 나머지 모든 컬럼을 범주형(Categorical)으로 변환하는 함수.
    """
    
    #   변환 대상 컬럼 찾기 (numeric_columns에 없는 컬럼들)
    categorical_columns = [col for col in train_df.drop(columns = ['임신 성공 여부']).columns if col not in numeric_columns]

    #   Categorical 변환
    train_df[categorical_columns] = train_df[categorical_columns].astype("object")
    test_df[categorical_columns] = test_df[categorical_columns].astype("object")

    print(f"  변환 완료! 범주형(object)으로 변환된 컬럼 개수: {len(categorical_columns)}개")
    
    return train_df, test_df

#   함수 실행
train, test = convert_to_categorical(train, test, numeric_columns)

In [ ]:
# 수정된 컬럼명으로 X, y 분리
X = train.drop(columns=['임신 성공 여부'])
y = train['임신 성공 여부']


# X에서 object 타입(범주형)인 컬럼 찾기
cat_columns = X.select_dtypes(include=['object']).columns.tolist()
X[cat_columns] = X[cat_columns].astype(str)
test[cat_columns] = test[cat_columns].astype(str)

# 결과 확인
print("  범주형 변수 목록:", cat_columns)

## 3.3. 튜닝

In [ ]:
#   데이터 타입 변환 (범주형 변수들을 category로 변환)
X_xgb = X.copy()
for col in X_xgb.select_dtypes(include=["object"]).columns:
    X_xgb[col] = X_xgb[col].astype("category")  #   범주형 변수를 category 타입으로 변환

#   Test 데이터 전처리 (범주형 변수 변환)
X_test_no_target = test.copy()  # Test 데이터 복사 (타겟 없음)
for col in X_test_no_target.select_dtypes(include=["object"]).columns:
    X_test_no_target[col] = X_test_no_target[col].astype("category")  #   범주형 변수를 category 타입으로 변환

In [ ]:
#   Stratified K-Fold 설정 (5-Fold CV)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

#   AUC 점수를 소수점 6자리까지 출력하는 custom scorer
def custom_auc_scorer(y_true, y_pred):
    auc_score = roc_auc_score(y_true, y_pred)
    rounded_score = round(auc_score, 6)  # 소수점 6자리 반올림
    print(f"  Fold AUC: {rounded_score}")  # Fold마다 출력
    return auc_score

#   XGBoost 하이퍼파라미터 검색 공간
param_grid = {
    "learning_rate": [0.036549283154481964],
    "n_estimators": [434],
    "max_depth": [4],
    "min_child_weight": [1],
    "gamma": [4],
    "subsample":[0.5879532027938977],
    "colsample_bytree": [0.7904350112349561],
    "lambda": [4],
    "alpha": [10],
    "scale_pos_weight": [6],
    "tree_method": ["hist"]
}

#   XGBoost 모델 정의
xgb_model = XGBClassifier(
    enable_categorical=True,
    random_state=42,
    use_label_encoder=False,
    verbosity=0
)


In [ ]:

#   Cross Validation 횟수 및 랜덤 시드 설정
n_cv = 3  # Cross Validation Fold 개수 설정 가능
cv_seed = 5534  # 랜덤 시드 설정

#   Stratified K-Fold 설정 (n_cv 사용)
cv = StratifiedKFold(n_splits=n_cv, shuffle=True, random_state=cv_seed)

#   AUC 점수를 소수점 6자리까지 출력하는 custom scorer
def custom_auc_scorer(y_true, y_pred):
    auc_score = roc_auc_score(y_true, y_pred)
    rounded_score = round(auc_score, 6)  # 소수점 6자리 반올림
    print(f"  Fold AUC: {rounded_score}")  # Fold마다 출력
    return auc_score

#   Bayesian Optimization 실행
start_time = time.time()

bayes_search = BayesSearchCV(
    xgb_model,
    param_grid,
    scoring=make_scorer(custom_auc_scorer, response_method="predict_proba"),
    cv=cv,  #   Cross Validation 횟수 적용 (n_cv)
    n_iter=1,  #   최대 30번 탐색
    n_jobs=30,  #   병렬 처리
    verbose=3,
    random_state=cv_seed  #   랜덤 시드 설정
)

# 💡 하이퍼파라미터 튜닝 실행
bayes_search.fit(X_xgb, y)

#   최적 파라미터 출력 (OrderedDict 형태로 변환)
best_params = OrderedDict(bayes_search.best_params_)
print(f"  XGBoost 최적 파라미터: {best_params}")

#   XGBoost n_cv-Fold Cross Validation 시작...
train_auc_scores = []
val_auc_scores = []

cv_start = time.time()

for fold_idx, (train_idx, val_idx) in tqdm(enumerate(cv.split(X, y), 1), total=n_cv, desc="🔄 XGBoost CV Progress"):
    X_t, X_v = X_xgb.iloc[train_idx], X_xgb.iloc[val_idx]
    y_t, y_v = y.iloc[train_idx], y.iloc[val_idx]

    #   최적 파라미터 적용한 모델 생성
    model = XGBClassifier(**best_params, enable_categorical=True, random_state=cv_seed)
    
    #   학습
    model.fit(X_t, y_t)

    #   Train AUC 계산
    y_train_pred = model.predict_proba(X_t)[:, 1]
    train_auc = roc_auc_score(y_t, y_train_pred)
    train_auc_scores.append(round(train_auc, 6))

    #   Validation AUC 계산
    y_val_pred = model.predict_proba(X_v)[:, 1]
    val_auc = roc_auc_score(y_v, y_val_pred)
    val_auc_scores.append(round(val_auc, 6))

cv_end = time.time()
cv_duration = cv_end - cv_start

#   XGBoost 결과 출력
print(f"\n  XGBoost {n_cv}-Fold Train AUC Scores = {train_auc_scores}")
print(f"  XGBoost {n_cv}-Fold Validation AUC Scores = {val_auc_scores}")

mean_train_auc = np.mean(train_auc_scores)
std_train_auc = np.std(train_auc_scores)

mean_val_auc = np.mean(val_auc_scores)
std_val_auc = np.std(val_auc_scores)

print(f"  XGBoost Train AUC: 평균={mean_train_auc:.6f}, 표준편차={std_train_auc:.6f}")
print(f"  XGBoost Validation AUC: 평균={mean_val_auc:.6f}, 표준편차={std_val_auc:.6f}")

total_time = time.time() - start_time
print(f"  XGBoost 학습 완료! 소요 시간: {total_time:.2f}초\n")

## 3.4. finalize

In [ ]:
#   1️⃣ 최적의 하이퍼파라미터 로드
best_params["objective"] = "binary:logistic"  # 이진 분류 문제
best_params["eval_metric"] = "auc"  # AUC 기준으로 성능 평가

#   2️⃣ 최적 하이퍼파라미터로 모델 생성
final_xgb_model = XGBClassifier(
    **best_params,
    enable_categorical=True,  #   범주형 데이터 직접 처리
    random_state=42,
    use_label_encoder=False,
    verbosity=0
)

#   3️⃣ 전체 데이터로 최종 모델 학습 (Validation 없이)
final_xgb_model.fit(X_xgb, y)

#   4️⃣ 모델 저장 (Pickle 파일로 저장하여 재사용 가능)
joblib.dump(final_xgb_model, dir_models + "final_xgb_model.pkl")
print("  최종 모델이 `final_xgb_model.pkl`로 저장되었습니다!")

## 3.5. predict

### 3.5.1. train predict

In [ ]:
with open(dir_models + "final_xgb_model.pkl", "rb") as f:
    final_xgb_model = pickle.load(f)

train_pred_xgb = final_xgb_model.predict_proba(X_xgb)[:,1]
pd.DataFrame(train_pred_xgb).to_csv(dir_to_submit + 'train_pred_xgb.csv', encoding = 'utf-8')

### 3.5.2. test predict

In [ ]:
#   3️⃣ 모델 예측 (확률 예측)
y_test_pred_proba = final_xgb_model.predict_proba(X_test_no_target)[:, 1]  # 클래스 1의 확률 예측
pd.DataFrame(y_test_pred_proba).to_csv(dir_to_submit + 'test_pred_xgb.csv', encoding = 'utf-8')

# 4. BERT

## 4.1. Preprocessing

- BERT는 따로 Preprocessing
- 기존 전처리와 동일하나 이상치만 제거

In [ ]:
train_df = pd.read_csv(dir + "train.csv",encoding='utf-8', index_col = 0)
test_df = pd.read_csv(dir + "test.csv",encoding='utf-8', index_col = 0)

In [ ]:
def remove_embryo_outliers(train_df):
    """
    배아 관련 이상치 데이터를 제거하는 함수.

    - 특정 시술 유형이 IVF이면서, 특정 시술 유형에 ICSI가 포함되지 않는데 미세 관련 변수가 0이 아닌 행 삭제
    - 특정 시술 유형이 IVF이면서, 이식된 배아 수가 0인데 배아 이식 경과일 > 0 인 행 삭제
    - 특정 시술 유형이 IVF이면서, 동결 배아 사용 여부가 0인데 해동된 배아 수 > 0인 행 삭제
    - 특정 시술 유형이 IVF이면서, 동결 배아 사용 여부가 0 & 배아 해동 경과일이 결측인데 해동된 배아 수 ≠ 0 인 행 삭제
    """

    # 삭제 전 데이터 크기 저장
    train_size_before = len(train_df)

    ###   1. 특정 시술 유형이 IVF이면서, 특정 시술 유형에 ICSI가 없는데 "미세" 포함 변수 값이 0이 아닌 행 삭제
    micro_columns = [col for col in train_df.columns if "미세" in col]  # "미세" 포함된 컬럼 찾기
    condition = (train_df["시술 유형"] == "IVF") & (~train_df["특정 시술 유형"].str.contains("ICSI", na=False)) & (train_df[micro_columns].sum(axis=1) > 0)
    train_df = train_df[~condition]

    ###   2. 특정 시술 유형이 IVF이면서, 이식된 배아 수 == 0인데 배아 이식 경과일 > 0 인 행 삭제
    condition = (train_df["시술 유형"] == "IVF") & (train_df["이식된 배아 수"] == 0) & (train_df["배아 이식 경과일"] > 0)
    train_df = train_df[~condition]

    ###   3. 특정 시술 유형이 IVF이면서, 동결 배아 사용 여부 == 0인데 해동된 배아 수 > 0 인 행 삭제
    condition = (train_df["시술 유형"] == "IVF") & (train_df["동결 배아 사용 여부"] == 0) & (train_df["해동된 배아 수"] > 0)
    train_df = train_df[~condition]

    ###   4. 특정 시술 유형이 IVF이면서, 동결 배아 사용 여부 == 0 & 배아 해동 경과일이 결측인데 해동된 배아 수 ≠ 0 인 행 삭제
    condition = (train_df["시술 유형"] == "IVF") & (train_df["동결 배아 사용 여부"] == 0) & (train_df["배아 해동 경과일"].isna()) & (train_df["해동된 배아 수"] != 0)
    train_df = train_df[~condition]

    # 삭제 후 데이터 크기 저장
    train_size_after = len(train_df)

    # 삭제된 행 수 계산
    train_removed = train_size_before - train_size_after
    #   삭제된 행 수 출력
    print(f"  Train 데이터에서 삭제된 행 수: {train_removed}")

    return train_df

train_df = remove_embryo_outliers(train_df)

In [ ]:
def remove_invalid_embryo_rows(train_df):
    """
    특정 시술 유형이 IVF인 경우,
    '동결 배아 사용 여부' == 0, '해동된 배아 수' == 0, '배아 해동 경과일'이 NaN이 아닌 경우의 행을 삭제하는 함수.
    """

    # 삭제 전 데이터 크기 저장
    train_size_before = len(train_df)

    #   특정 시술 유형이 IVF인 경우만 적용
    condition = (train_df['시술 유형'] == "IVF") & \
                (train_df['동결 배아 사용 여부'] == 0) & \
                (train_df['해동된 배아 수'] == 0) & \
                (pd.notna(train_df['배아 해동 경과일']))  #   NaN이 아닌 경우 필터링
    train_df = train_df[~condition]  #   조건에 맞는 행 삭제

    # 삭제된 행 수 계산
    train_removed = train_size_before - len(train_df)

    #   삭제된 행 수 출력
    print(f"  Train 데이터에서 삭제된 행 수: {train_removed}")

    return train_df

#   함수 사용 예시
train_df = remove_invalid_embryo_rows(train_df)

In [ ]:
def remove_outliers_procedure(train_df):
    """
    '특정 시술 유형' 컬럼이 존재할 경우, 결측(NaN)인 행을 제거하고, 제거된 행 수를 출력하는 함수.
    컬럼이 없으면 경고 메시지를 출력하고 원래 데이터프레임을 반환.
    """

    # 특정 시술 유형 컬럼 존재 여부 확인
    if '특정 시술 유형' not in train_df.columns:
        print("⚠️ '특정 시술 유형' 컬럼이 데이터프레임에 존재하지 않습니다. 원본 데이터를 반환합니다.")
        return train_df

    # 원본 데이터 크기 저장
    train_size_before = len(train_df)

    # '특정 시술 유형'이 NaN인 행 제거
    train_df = train_df.dropna(subset=['특정 시술 유형'])

    # 제거된 행 수 계산
    train_removed = train_size_before - len(train_df)

    # 제거된 행 수 출력
    print(f"  Train 데이터에서 제거된 행 수: {train_removed}")

    return train_df
train_df = remove_outliers_procedure(train_df)

In [ ]:
def fill_missing_embryo_days(train_df, test_df):
    """
    IVF 시술을 받은 환자의 경우,
    - '이식된 배아 수'가 0이면 '배아 이식 경과일'의 결측값을 0으로 채움.
    - '동결 배아 사용 여부'가 0이면 '배아 해동 경과일'의 결측값을 0으로 채움.

    train_df와 test_df 모두 동일한 로직을 적용함.
    """

    def fill_embryo_days(df):
        #   IVF 시술을 받은 경우만 적용
        ivf_mask = df['시술 유형'] == "IVF"

        #   '이식된 배아 수' == 0 → '배아 이식 경과일' NaN을 0으로 채우기
        condition_1 = ivf_mask & (df['이식된 배아 수'] == 0) & (df['배아 이식 경과일'].isna())
        df.loc[condition_1, '배아 이식 경과일'] = 0

        #   '동결 배아 사용 여부' == 0 → '배아 해동 경과일' NaN을 0으로 채우기
        condition_2 = ivf_mask & (df['동결 배아 사용 여부'] == 0) & (df['배아 해동 경과일'].isna())
        df.loc[condition_2, '배아 해동 경과일'] = 0

        #   변경된 데이터 개수 출력
        print(f"  [데이터셋: {df.name}] '배아 이식 경과일' 채운 개수: {condition_1.sum()}")
        print(f"  [데이터셋: {df.name}] '배아 해동 경과일' 채운 개수: {condition_2.sum()}")

        return df

    #   DataFrame 이름 추가 (출력 시 구분하기 위해)
    train_df.name = "Train"
    test_df.name = "Test"

    train_df = fill_embryo_days(train_df)
    test_df = fill_embryo_days(test_df)

    return train_df, test_df


#   함수 실행
train_df, test_df = fill_missing_embryo_days(train_df, test_df)

In [ ]:
print(train_df.shape); print(test_df.shape)

## 4.2. feature tokenization

In [ ]:
#   Fast Tokenizer 사용하여 속도 향상 및 RAM 절약
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

#   컬럼 정의
category_columns = [
    "시술 시기 코드", "시술 당시 나이", "시술 유형", "배란 유도 유형", "배아 생성 주요 이유",
    "총 시술 횟수", "클리닉 내 총 시술 횟수", "IVF 시술 횟수", "DI 시술 횟수",
    "총 임신 횟수", "IVF 임신 횟수", "DI 임신 횟수", "총 출산 횟수",
    "IVF 출산 횟수", "DI 출산 횟수", "난자 출처", "정자 출처", "난자 기증자 나이", "정자 기증자 나이"
]

binary_columns = [
    "배란 자극 여부", "단일 배아 이식 여부", "착상 전 유전 검사 사용 여부", "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인", "남성 부 불임 원인", "여성 주 불임 원인", "여성 부 불임 원인",
    "부부 주 불임 원인", "부부 부 불임 원인", "불명확 불임 원인", "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인", "불임 원인 - 배란 장애", "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증", "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성", "불임 원인 - 정자 형태",
    "동결 배아 사용 여부", "신선 배아 사용 여부", "기증 배아 사용 여부", "대리모 여부",
    "PGD 시술 여부", "PGS 시술 여부"
]

numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수", "총 생성 배아 수", "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수", "이식된 배아 수", "미세주입 배아 이식 수",
    "저장된 배아 수", "미세주입 후 저장된 배아 수", "해동된 배아 수",
    "해동 난자 수", "수집된 신선 난자 수", "저장된 신선 난자 수",
    "혼합된 난자 수", "파트너 정자와 혼합된 난자 수", "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일", "난자 해동 경과일", "난자 혼합 경과일",
    "배아 이식 경과일", "배아 해동 경과일"
]

#   결측값 처리 (RAM 절약)
train_df.fillna({"시술 시기 코드": "알 수 없음", "배란 자극 여부": 0}, inplace=True)
test_df.fillna({"시술 시기 코드": "알 수 없음", "배란 자극 여부": 0}, inplace=True)

for col in numeric_columns:
    median_value = train_df[col].median()
    train_df[col] = train_df[col].fillna(median_value)
    test_df[col] = test_df[col].fillna(median_value)

#   연속형 변수 Binning (필요할 때만 실행)
for col in numeric_columns:
    train_df[col] = pd.qcut(train_df[col], q=10, duplicates="drop", labels=False)
    test_df[col] = pd.qcut(test_df[col], q=10, duplicates="drop", labels=False)


#   Feature Tokenization 수행
def tokenize_features(df):
    return [
        ", ".join(
            [f"{col}: {row[col]}" for col in category_columns] +
            [f"{col}: {'True' if row[col] == 1 else 'False'}" for col in binary_columns] +
            [f"{col}: 구간 {row[col]}" for col in numeric_columns]
        )
        for _, row in df.iterrows()
    ]

train_texts = tokenize_features(train_df)
test_texts = tokenize_features(test_df)

#   Tokenization을 배치 단위로 수행하여 RAM 절약
def batch_tokenize(texts, tokenizer, batch_size=128):
    input_ids_list, attention_mask_list = [], []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        encodings = tokenizer(batch_texts, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
        input_ids_list.append(encodings["input_ids"])
        attention_mask_list.append(encodings["attention_mask"])

    return torch.cat(input_ids_list), torch.cat(attention_mask_list)

train_input_ids, train_attention_masks = batch_tokenize(train_texts, tokenizer, batch_size=128)
test_input_ids, test_attention_masks = batch_tokenize(test_texts, tokenizer, batch_size=128)

#   NumPy 배열을 사용하여 RAM 절약
train_input_ids = train_input_ids.to(torch.int32)
test_input_ids = test_input_ids.to(torch.int32)

#   불필요한 데이터 삭제하여 RAM 확보
del train_texts, test_texts
gc.collect()

In [ ]:
#   Target 데이터 변환 (Test 데이터에는 `labels` 없음!)
train_labels = torch.tensor(train_df["임신 성공 여부"].values, dtype=torch.long)

#   Tokenized 데이터 저장
torch.save({"input_ids": train_input_ids, "attention_mask": train_attention_masks, "labels": train_labels}, dir + "train_data_fixed.pt")
torch.save({"input_ids": test_input_ids, "attention_mask": test_attention_masks}, dir + "test_data_fixed.pt")  # ❌ labels 없음!

print("  Tokenized 데이터 저장 완료!")

#   DataLoader 변환
BATCH_SIZE = 16

train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

print("  Tokenized Train Shape:", train_input_ids.shape)
print("  Tokenized Test Shape:", test_input_ids.shape)
print("  Train Labels Shape:", train_labels.shape)

## 4.3. tokenized 데이터 불러오기

In [ ]:
#   저장된 Train 데이터 불러오기
train_data = torch.load(dir + "train_data_fixed.pt")
train_input_ids = train_data["input_ids"]
train_attention_masks = train_data["attention_mask"]
train_labels = train_data["labels"]

#   저장된 Test 데이터 불러오기 (`labels` 없음)
test_data = torch.load(dir + "test_data_fixed.pt")
test_input_ids = test_data["input_ids"]
test_attention_masks = test_data["attention_mask"]

print("  저장된 Tokenized 데이터 불러오기 완료!")

#   Train DataLoader 생성
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

#   Test DataLoader 생성 (labels 없음!)
test_dataset = TensorDataset(test_input_ids, test_attention_masks)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

print("  DataLoader 생성 완료!")

trian validation split

In [ ]:
#   Train / Validation 분할 (80:20)
train_input_ids, val_input_ids, train_attention_masks, val_attention_masks, train_labels, val_labels = train_test_split(
    train_data["input_ids"], train_data["attention_mask"], train_data["labels"], test_size=0.2, random_state=42
)

BATCH_SIZE = 16

train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("  Train/Validation 데이터 분할 완료!")

## 4.4. 모델 학습


In [ ]:


#   GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#   DistilBERT 모델 사용 (속도 2배 증가)
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
model.to(device)
model.gradient_checkpointing_enable()  #   VRAM 절약

#   옵티마이저 및 스케줄러
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
num_training_steps = len(train_dataloader) * 5
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

#   Mixed Precision Training (AMP)
scaler = GradScaler()

#   학습 파라미터
EPOCHS = 5
best_auc = 0.0
gradient_accumulation_steps = 4  #   작은 배치를 모아서 업데이트

#   학습 루프
for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    optimizer.zero_grad()

    # Training Loop
    for step, batch in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{EPOCHS} Training")):
        input_ids, attention_mask, labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)

        with autocast():
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss / gradient_accumulation_steps  #   작은 배치 단위로 나눔

        scaler.scale(loss).backward()

        if (step + 1) % gradient_accumulation_steps == 0:  #   배치 누적 후 최적화 실행
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        train_loss += loss.item()

    print(f"Epoch {epoch+1} Train Loss: {train_loss / len(train_dataloader):.4f}")

    #   Validation 단계 (AUC 계산)
    model.eval()
    val_loss = 0
    true_labels = []
    pred_probs = []

    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc=f"Epoch {epoch+1}/{EPOCHS} Validation"):
            input_ids, attention_mask, labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            val_loss += loss.item()

            #   AUC 점수 계산을 위한 확률값 저장
            probs = torch.softmax(logits, dim=1)[:, 1]  #   임신 성공 (1)의 확률
            true_labels.extend(labels.cpu().numpy())
            pred_probs.extend(probs.cpu().numpy())

    avg_val_loss = val_loss / len(val_dataloader)
    val_auc = roc_auc_score(true_labels, pred_probs)

    print(f"Epoch {epoch+1} Validation Loss: {avg_val_loss:.4f}, AUC: {val_auc:.4f}")

    #   AUC가 가장 높은 모델 저장
    if val_auc > best_auc:
        best_auc = val_auc
        torch.save(model.state_dict(), dir_models + "best_bert_model_fixed.pth")
        print("  Best Model Saved!")

## 4.5. Prediction

### 4.5.1. train 예측

In [ ]:
#   저장된 Train 데이터 불러오기
train_data = torch.load(dir + "train_data_fixed.pt")
train_input_ids = train_data["input_ids"]
train_attention_masks = train_data["attention_mask"]
train_labels = train_data["labels"]

#   저장된 Test 데이터 불러오기 (`labels` 없음)
test_data = torch.load(dir + "test_data_fixed.pt")
test_input_ids = test_data["input_ids"]
test_attention_masks = test_data["attention_mask"]

print("  저장된 Tokenized 데이터 불러오기 완료!")

#   Train DataLoader 생성
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

#   Test DataLoader 생성 (labels 없음!)
test_dataset = TensorDataset(test_input_ids, test_attention_masks)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

print("  DataLoader 생성 완료!")

In [ ]:
#   GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


#   저장된 모델 불러오기
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
model.load_state_dict(torch.load(dir_models + "best_bert_model_fixed.pth", map_location=device))  # 모델 가중치 로드
model.to(device)
model.eval()  # 평가 모드 설정

print("  최적의 DistilBERT 모델 불러오기 완료!")

#   Train 데이터 예측 함수
def predict(model, dataloader):
    predictions = []
    pred_probs = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Predicting"):
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            probs = torch.softmax(logits, dim=1)  #   확률값 (Softmax)
            preds = torch.argmax(probs, dim=1)  #   최종 예측값 (0 or 1)

            predictions.extend(preds.cpu().numpy())
            pred_probs.extend(probs[:, 1].cpu().numpy())  #   "임신 성공(1)" 확률 저장

    return np.array(predictions), np.array(pred_probs)

#   Train 데이터 예측 수행
train_predictions, train_probabilities = predict(model, train_dataloader)

print("  Train 데이터 예측 완료!")


In [ ]:
# train 예측 저장
pd.DataFrame({'predictions':train_predictions, 'probability': train_probabilities}).to_csv(dir_to_submit + "train_BERT_probability_fixed.csv")

### 4.5.2. test 예측

In [ ]:
#   GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#   저장된 모델 경로
best_model_path = dir_models + "best_bert_model_fixed.pth"  # 저장된 모델 파일

#   DistilBERT 모델 불러오기 (저장된 모델과 동일한 모델 사용)
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
model.load_state_dict(torch.load(best_model_path, map_location=device))  # 가중치 로드
model.to(device)
model.eval()  # 평가 모드 설정

print("  최적의 DistilBERT 모델 불러오기 완료!")

In [ ]:
#   저장된 Test 데이터 불러오기 (labels 없음!)
test_data = torch.load(dir + "test_data_fixed.pt")
test_input_ids = test_data["input_ids"]
test_attention_masks = test_data["attention_mask"]


BATCH_SIZE = 16
test_dataset = TensorDataset(test_input_ids, test_attention_masks)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("  Test 데이터 불러오기 완료!")

In [ ]:

#   테스트 데이터 예측 함수
def predict(model, dataloader):
    predictions = []
    pred_probs = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Predicting"):
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            probs = torch.softmax(logits, dim=1)  #   확률값 (Softmax)
            preds = torch.argmax(probs, dim=1)  #   최종 예측값 (0 or 1)

            predictions.extend(preds.cpu().numpy())
            pred_probs.extend(probs[:, 1].cpu().numpy())  #   "임신 성공"일 확률 저장

    return np.array(predictions), np.array(pred_probs)

#   Test 데이터 예측 수행
test_predictions, test_probabilities = predict(model, test_dataloader)

print("  Test 데이터 예측 완료!")

In [ ]:
sample_submission = pd.read_csv(dir + 'sample_submission.csv', encoding = 'utf-8')
sample_submission['predictions'] = test_predictions
sample_submission['probability'] = test_probabilities
display(sample_submission)

In [ ]:
sample_submission.to_csv(dir_to_submit + "BERT_probability_fixed.csv", encoding= 'utf-8')

-------

# 5. Stacking 

* Base Model : CatBoost, XgBoost, BERT

    * Base Model 1 : CatBoost

    * Base Model 2 : XgBoost

    * Base Model 3 : BERT

* Meta Model : Logistic Regressor

In [ ]:
# train data load
train_cat = pd.read_csv('./submit/train_pred_cat.csv', encoding='utf-8').iloc[:,1]
train_xgb = pd.read_csv('./submit/train_pred_xgb.csv', encoding='utf-8').iloc[:,1]
train_bert = pd.read_csv('./submit/train_BERT_probability_fixed.csv', encoding='utf-8')['probability']

In [ ]:
# test data load
test_cat = pd.read_csv('./submit/test_pred_cat.csv', encoding = 'utf-8').iloc[:,1]
test_xgb = pd.read_csv('./submit/test_pred_xgb.csv', encoding = 'utf-8').iloc[:,1]
test_bert = pd.read_csv('./submit/BERT_probability_fixed.csv', encoding = 'utf-8')['probability']

In [ ]:
# shape 확인 (254733,)(90067,)
print(train_cat.shape,end = ''); print(test_cat.shape)
print(train_xgb.shape,end = ''); print(test_xgb.shape)
print(train_bert.shape,end = ''); print(test_bert.shape)

In [ ]:
# stack : catboost, xgboost, bert
stack_train = np.column_stack((train_cat,train_xgb,train_bert))
stack_test = np.column_stack((test_cat,test_xgb, test_bert))
print(stack_train.shape)
print(stack_test.shape)

#   메타 모델 (Logistic Regression) 학습
meta_model = LogisticRegression()
meta_model.fit(stack_train, y)

#   최종 예측 수행
final_predictions = meta_model.predict_proba(stack_test)[:, 1]

#   예측값 출력
print("\n  stack_test Predictions Completed!")
print(final_predictions[:10])  # 앞 10개 예측값 출력 (샘플)

In [ ]:
meta_model.coef_

In [ ]:
sample_submission = pd.read_csv(dir + 'sample_submission.csv', encoding = 'utf-8')
sample_submission['probability'] = final_predictions
display(sample_submission)

In [ ]:
# 제출할 파일명
file_name = dir_to_submit + "final_submission.csv"
sample_submission.to_csv(file_name, index=False, encoding = 'utf-8')

In [ ]:
sample_submission